In [ ]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
import webbrowser
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut



vehicles_df = pd.read_csv('vehicles_df.csv')

# Aggregate data by city
city_counts = vehicles_df.groupby('City').size().reset_index(name='Count')

# Geocode cities to get latitude and longitude
geolocator = Nominatim(user_agent="geoapi")
city_coordinates = []

for city in city_counts['City'].unique():
    try:
        location = geolocator.geocode(f"{city}, Illinois")  # Update state/region as needed
        if location:
            city_coordinates.append({
                "City": city,
                "latitude": location.latitude,
                "longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": city,
                "latitude": None,
                "longitude": None
            })
    except GeocoderTimedOut:
        city_coordinates.append({
            "City": city,
            "latitude": None,
            "longitude": None
        })
# Convert geocoded coordinates to DataFrame and merge with city counts
city_coordinates_df = pd.DataFrame(city_coordinates)
city_data = pd.merge(city_counts, city_coordinates_df, on="City", how="left")

# Filter out cities with missing coordinates
city_data = city_data.dropna(subset=["latitude", "longitude"])

# Initialize Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
        # Title and Introduction
    html.Div([
        html.H1("Vehicle Hierarchy Dashboard", style={'textAlign': 'center'}),
        html.P(
            "This dashboard provides an interactive view of vehicle registrations, categorized by their "
            "make, model, and color. Use the dropdown below to filter by fuel type and explore the hierarchy.",
            style={'textAlign': 'center', 'fontSize': '18px', 'marginBottom': '20px'}
        ),
        html.P(
            "Hover over the segments in the sunburst chart to view detailed information about the vehicles.",
            style={'textAlign': 'center', 'fontSize': '16px', 'color': 'white'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#769199'}),
   

        html.Div([
        html.P(
            "This dashboard is for demonstration purposes only and is not connected to a live database. "
            "The data used here is a sample dataset and does not reflect actual vehicle registrations.",
            style={'textAlign': 'center', 'fontSize': '14px', 'color': 'white'}
        )
    ], style={'padding': '20px', 'backgroundColor': '#769199'})
])

# Callback for Sunburst, Line, and City Map



# Run the App
if __name__ == '__main__':
    webbrowser.open_new('http://127.0.0.1:3000/')
    app.run_server(debug=True, host='127.0.0.1', port=3000)

